# Notebook for Demand forecasting

## Imports

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#import seaborn as sns
#sns.set_context("paper", font_scale=1.3)
#sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn import preprocessing
from sklearn.feature_selection import f_regression, SelectKBest

from sklearn.model_selection import train_test_split

from numpy.random import seed
seed(20)
from sklearn.metrics._scorer import make_scorer

pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True)

import tensorflow as tf

## Helper Methods 

In [6]:
import logging
from scipy.stats import pearsonr

# define the behavior of output stream: into the console + into a file
class Logger:
    def __init__(self, filename):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        
        file_handler = logging.FileHandler(filename)
        file_handler.setLevel(logging.INFO)
        
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        
        self.logger.addHandler(file_handler)

    def log(self, message):
        self.logger.info(message)

import sys

# Calculate pearson coefficient
def spcc(y_true, y_pred, **kwargs):
    corr, _ = pearsonr(y_true, y_pred)
    return corr
    
# create a slicing window dataset (each window is a sequence of data points that can be used for prediction
# Prepare dataset for model training 
# No Direct Modification
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    #dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
# batch (group) windows for more efficient computing
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset
    
#Takes a dataframe with the holiday field and returns encoded dataframe.
def onehotholiday(select):
    X_2 = select[['Holiday']]
    # Create a OneHotEncoder object
    enc = preprocessing.OneHotEncoder(sparse_output=False)
    
    # Fit and transform
    onehotlabels = enc.fit_transform(X_2)
    
    # Create column names
    column_values = [f'Holiday_{i}' for i in range(onehotlabels.shape[1])]

    # Create the dataframe 
    onehotholiday = pd.DataFrame(data=onehotlabels, columns=column_values)

    # Join with original dataset
    dataset = select.drop(columns=['Holiday'])
    dataset = dataset.join(onehotholiday)
    
    # Reorder columns to put '2to5' at the end
    cols = [col for col in dataset.columns if col != '2to5'] + ['2to5']
    dataset = dataset[cols]
    
    return dataset
    
# Create a dataframe contains past 7(look_back) days sales (a numpy array) as a feature
def add_lookback(dataset, look_back, df):
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]['2to5']
        a = a.values
        for j in range(len(a)):
            df[j][i]= a[j]
    return df

## Data Preprocessing

In [7]:
np.random.seed(42)
# fix random seed for reproducibility
# load the dataset
dataframe = pd.read_csv('./RestaurantDataVets_All_2to5 (1).csv')
data = dataframe.drop(columns=['Index','Group','DMY','MissingPrevDays','DailyAvg','DailyBusyness'])

lookback=20
dataframe_removed_lookback = data.drop([x for x in range(lookback)])

for i in range(lookback):
    dataframe_removed_lookback[i] = 1.0
    
df = dataframe_removed_lookback[['Year', 'Day', 'January','February',
                         'March','April','May','June','July',
                         'August', 'September', 'October', 'November',
                         'December','Sunday', 'Monday', 'Tuesday',
                         'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Holiday', 'Carnival', 
                         'LentFasting','Ramadan','ChristmasSeason',
                         'WeeklyAvg','MinSales','MaxSales',
                         'WeeklyBusyness',
                         0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                          14, 15, 16, 17, 18, 19,      
                         '2to5']]
df = df.reset_index(drop=True)

#Objects need to be converted to float due to missing values at load time.
#df["DailyAvg"] = df.DailyAvg.astype(float)
df["WeeklyAvg"] = df.WeeklyAvg.astype(float)
df["MinSales"] = df.MinSales.astype(float)
df["MaxSales"] = df.MaxSales.astype(float)
#df["DailyBusyness"] = df.DailyBusyness.astype(float)
df["WeeklyBusyness"] = df.WeeklyBusyness.astype(float)

lb_data = add_lookback(data, lookback, df)
lb_data = lb_data.reset_index(drop=True)

hotdata = onehotholiday(lb_data)
hotdata = hotdata.drop(columns=[14,15,16,17,18,19])

hot_numcols = len(hotdata.columns)
dataset = hotdata.values

lbset=lb_data.values
lb_numcols =  len(lb_data.columns)

print("train_df Shape:" ,lb_data.shape)
print("After encoding:", hotdata.shape)

X=dataset[:, 0:hot_numcols-1]
y=lbset[:, lb_numcols-7:lb_numcols]

scaler = preprocessing.RobustScaler()
X = scaler.fit_transform(X,y)

sys.stdout.flush()

scoring = {
        'mae': 'neg_mean_absolute_error',
        'mse' : 'neg_mean_squared_error',
        'custom': make_scorer(spcc, greater_is_better=True)
    }

j_arr = []
j=70
feat_reduction = SelectKBest(f_regression, k=j+1) 
X_new = feat_reduction.fit_transform(X,y[:,0])
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=221, random_state=42, shuffle=False)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
            
# meansq = keras.metrics.mean_squared_error(y_test.flatten(), y_pred.flatten()).numpy()
# meanabs = keras.metrics.mean_absolute_error(y_test.flatten(), y_pred.flatten()).numpy()
# print("MSE = "+str(meansq))
# print("MAE = "+str(meanabs))




train_df Shape: (1091, 51)
After encoding: (1091, 72)
(870, 71) (870, 7) (221, 71) (221, 7)


## Actual Run

In [24]:
import torch
import torchvision

# Use a transformer model for sequential predicting
class PredictModel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.preprocess=torch.nn.Linear(1,128)
        self.model=torch.nn.Transformer(d_model=128, nhead=1, num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=128, dropout=0.1, activation='gelu')
        self.output_layer = torch.nn.Linear(128,7)
    
    def forward(self, x,out_feature=False):
        x=torch.reshape(x,(x.shape[0],x.shape[1],1))
        x=self.preprocess(x)
        x = self.model(x,x)
        feature=x[:,0,:]
        x = self.output_layer(feature)
        if out_feature:
            return x,feature
        return x

# Use an LSTM model
class LSTMModel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.lstm=torch.nn.LSTM(1,128,3)
        self.output_layer = torch.nn.Linear(128,7)
    
    def forward(self, x):
        x=torch.reshape(x,(x.shape[0],x.shape[1],1))
        x,_=self.lstm(x)
        x=x[:,-1,:]
        x = self.output_layer(x)
        return x
    
# Use a GRU model
class GRUModel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.gru=torch.nn.GRU(1,128,3)
        self.output_layer = torch.nn.Linear(128,7)
    
    def forward(self, x):
        x=torch.reshape(x,(x.shape[0],x.shape[1],1))
        x,_=self.gru(x)
        x=x[:,-1,:]
        x = self.output_layer(x)
        return x
    
model=PredictModel()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn=torch.nn.MSELoss()

batch_size=16
batches=X_train.shape[0]//batch_size
for epoch in range(1):
    model=model.train()
    for batch_id in range(batches):
        x=torch.tensor(X_train[batch_id*batch_size:(batch_id+1)*batch_size]).float()
        y=torch.tensor(y_train[batch_id*batch_size:(batch_id+1)*batch_size]).float()
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print("The loss is: ",loss.item())

    # Test the performance
    model=model.eval()
    test_batches=X_test.shape[0]//batch_size
    for batch_id in range(test_batches):
        x=torch.tensor(X_test[batch_id*batch_size:(batch_id+1)*batch_size]).float()
        y=torch.tensor(y_test[batch_id*batch_size:(batch_id+1)*batch_size]).float()
        x=torch.tensor(x).float()
        y=torch.tensor(y).float()
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        print("The test loss is: ",loss.item())

The loss is:  331146.15625
The loss is:  290660.8125
The loss is:  585638.6875
The loss is:  605075.4375
The loss is:  920085.625
The loss is:  647891.3125
The loss is:  725590.6875
The loss is:  830078.8125
The loss is:  784741.1875
The loss is:  713534.3125
The loss is:  630002.625
The loss is:  800105.5625
The loss is:  625068.5
The loss is:  783487.9375
The loss is:  538357.625
The loss is:  629568.375
The loss is:  607629.0625
The loss is:  667380.5625
The loss is:  687239.6875
The loss is:  750077.6875
The loss is:  790963.8125
The loss is:  639960.0625
The loss is:  701708.6875
The loss is:  659331.125
The loss is:  716275.3125
The loss is:  631751.8125
The loss is:  769635.9375
The loss is:  715487.375
The loss is:  654364.625
The loss is:  1679842.125
The loss is:  1782179.375
The loss is:  1886618.625
The loss is:  1645270.125
The loss is:  1800364.0
The loss is:  1645276.0
The loss is:  1572248.875
The loss is:  1409131.0
The loss is:  1325379.375
The loss is:  1456026.0
The

In [25]:
# Use the trained model to get features
features_X_train=[]
features_X_test=[]

model=model.eval()
batch_size=16
batches=X_train.shape[0]//batch_size
for batch_id in range(batches):
    x=torch.tensor(X_train[batch_id*batch_size:(batch_id+1)*batch_size]).float()
    y=torch.tensor(y_train[batch_id*batch_size:(batch_id+1)*batch_size]).float()
    y_pred,feature=model(x,out_feature=True)
    feature=feature.detach().numpy()
    for i in range(feature.shape[0]):
        features_X_train.append(feature[i])
features_X_train=np.array(features_X_train)

test_batches=X_test.shape[0]//batch_size
for batch_id in range(test_batches):
    x=torch.tensor(X_test[batch_id*batch_size:(batch_id+1)*batch_size]).float()
    y=torch.tensor(y_test[batch_id*batch_size:(batch_id+1)*batch_size]).float()
    y_pred,feature=model(x,out_feature=True)
    feature=feature.detach().numpy()
    for i in range(feature.shape[0]):
        features_X_test.append(feature[i])
features_X_test=np.array(features_X_test)

print(features_X_train.shape,features_X_test.shape)


(864, 128) (208, 128)


In [27]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from tensorflow import keras
neigh = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=7))
y_train=y_train[:features_X_train.shape[0],...]
neigh.fit(features_X_train, y_train)
y_test=y_test[:features_X_test.shape[0],...]
y_pred = neigh.predict(features_X_test)
            
meansq = keras.metrics.mean_squared_error(y_test.flatten(), y_pred.flatten()).numpy()
meanabs = keras.metrics.mean_absolute_error(y_test.flatten(), y_pred.flatten()).numpy()
print("MSE = "+str(meansq))
print("MAE = "+str(meanabs))

ValueError: X has 71 features, but KNeighborsRegressor is expecting 128 features as input.